In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np

matplotlib.rcParams.update({'figure.figsize':(14,10), 'font.size':14})

/Users/efuentes/.virtualenvs/irReceiver/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [154]:
dataFolder = 'data'
files = [x for x in os.listdir(dataFolder) if x.endswith('.csv')]
print files

['0.csv', '0Rep.csv', '1.csv', '1Rep.csv', '2.csv', '2Rep.csv', '3.csv', '3Rep.csv', '4.csv', '4Rep.csv', '5.csv', '5Rep.csv', '6.csv', '6Rep.csv', '7.csv', '7Rep.csv', '8.csv', '8Rep.csv', '9.csv', '9Rep.csv', 'chdown.csv', 'chdownRep.csv', 'chup.csv', 'chupRep.csv', 'enter.csv', 'enterRep.csv', 'info.csv', 'infoRep.csv', 'lang.csv', 'langRep.csv', 'last.csv', 'lastRep.csv', 'mute.csv', 'muteRep.csv', 'power.csv', 'powerRep.csv', 'voldown.csv', 'voldownRep.csv', 'volup.csv', 'volupRep.csv']


In [155]:
traces = {}
for f in files:
    fName = os.path.join(dataFolder, f)
    with open(fName) as fileIn:
        key = f.split('.')[0]
        d = np.loadtxt(fName, skiprows=1, delimiter=',')
        leadingSelect = d[:,1] == 0
        leading = d[leadingSelect, 0]
        traces[key] = leading

In [175]:
deltas = []

def processPacket(diffs, verbose=False):
    if verbose:
        print 'Diffs:'
        print diffs
    
    # Sanity Checking
    if len(diffs) != 17:
        if verbose:
            print '!!!!!!!!! Wrong Length'
        return None

    timingInfo = diffs[:8]
    buttonInfo = diffs[9:]
    
    minSpace = diffs[2]
    maxSpace = diffs[3]
    halfSpaces = diffs[4:6]
    oneBelowHalfSpace = diffs[6]
    oneBelowFullSpace = diffs[7]
    
    if (maxSpace - np.sum(halfSpaces) > .1 or # if the sum of half spaces isn't within 100 microseconds
        oneBelowFullSpace - halfSpaces[-1] - oneBelowHalfSpace > .1):
        if verbose:
            print '!!!!!!!!! Invalid Timing Packet'
        return None    
    
    spacing = (maxSpace - minSpace) / 15
    
    buttonLevels = buttonInfo / spacing
    if verbose:
        print 'Button Levels:'
        print buttonLevels
    if np.max(np.abs((buttonLevels - np.round(buttonLevels)))) > .4: # If any of the levels are close to being 
                                                                     # undecideable, skip it
        if verbose:
            print '!!!!!!!!! Undecideable Button Info'
        return None

    buttonDetectedLevels = np.round(buttonLevels) - 7
    if verbose:
        print 'Quantized Button Levels:'
        print buttonDetectedLevels
    if np.max(buttonDetectedLevels) > 15 or np.min(buttonDetectedLevels) < 0:
        if verbose:
            print '!!!!!!!!! Invalid Button Level'
        return None
    return buttonDetectedLevels

def decodeKey(levels):
    # Sanity Checks
    if np.sum(levels) % 16 != 0:
        return None, False
    if len(levels) != 8:
        return None, False

    isRepeat = False
    if levels[2] == 8:
        isRepeat = True
    
    ret = None
    val = levels[5]
    if levels[4] == 0:
        if val < 10:
            ret = str(val)
        elif val == 10:
            ret = 'volup'
        elif val == 11:
            ret = 'voldown'
        elif val == 12:
            ret = 'mute'
        elif val == 13:
            ret = 'chup'
        elif val == 14:
            ret = 'chdown'
        elif val == 15:
            ret = 'power'
    else:
        if val == 1:
            ret = 'last'
        if val == 2:
            ret = 'lang'
        if val == 5:
            ret = 'enter'
        if val == 6:
            ret = 'info'

    return ret, isRepeat
    
    pass
#     print levels

residuals = np.array([])
buttonDict = {}
for key in [x for x in sorted(traces.keys()) if 'Rep' not in x]:
#     print 'Key:', key
#     print len(traces[key])
    for i in range(0, len(traces[key]) / 18):
        leading = traces[key][i * 18:(i + 1) * 18]
        diffs = np.diff(leading) * 1000 # convert to ms
        levels = processPacket(diffs)
        buttonDict[key] = levels
        print decodeKey(levels)

#         spacing = (diffs[4] + diffs[5]) / 2.0 - diffs[6]
#         print 'Key:',key
#         print diffs[:8]

#         minSpace = diffs[2]
#         maxSpace = diffs[3]
#         deltaSpace = diffs[5] - diffs[6]
#         print deltaSpace

#         print maxSpace - minSpace

#         print (maxSpace - minSpace) / deltaSpace

#         data = diffs[9:] / (maxSpace - minSpace) * 15
#     #     data = diffs[9:] / deltaSpace
#         print data
#         residuals = np.hstack([data - np.round(data), residuals])
    
# _ = plt.hist(residuals, 20)
for k in sorted(buttonDict.keys()):
    print ' '.join(['{0:2d} '.format(int(x)) for x in buttonDict[k]]), k

('0.0', False)
('1.0', False)
('2.0', False)
('3.0', False)
('4.0', False)
('5.0', False)
('6.0', False)
('7.0', False)
('8.0', False)
('9.0', False)
('chdown', False)
('chup', False)
('enter', False)
('info', False)
('lang', False)
('last', False)
('mute', False)
('power', False)
('voldown', False)
('volup', False)
 1  15   0   0   0   0   0   0  0
 1  14   0   0   0   1   0   0  1
 1  13   0   0   0   2   0   0  2
 1  12   0   0   0   3   0   0  3
 1  11   0   0   0   4   0   0  4
 1  10   0   0   0   5   0   0  5
 1   9   0   0   0   6   0   0  6
 1   8   0   0   0   7   0   0  7
 1   7   0   0   0   8   0   0  8
 1   6   0   0   0   9   0   0  9
 1   1   0   0   0  14   0   0  chdown
 1   2   0   0   0  13   0   0  chup
 1   8   0   0   2   5   0   0  enter
 1   7   0   0   2   6   0   0  info
 1   5   0   0   8   2   0   0  lang
 1   9   0   0   5   1   0   0  last
 1   3   0   0   0  12   0   0  mute
 1   0   0   0   0  15   0   0  power
 1   4   0   0   0  11   0   0  voldown
 1

In [176]:

for key in sorted([x for x in traces.keys() if 'Rep' in x]):
    print 'Key:', key
    print 'Num Packets:', len(traces[key]) / 18
    for i in range(0, len(traces[key]) / 18):

        leading = traces[key][i*18:(i+1)*18]
        diffs = np.diff(leading) * 1000 # convert to ms
        levels = processPacket(diffs)
        print levels
        print decodeKey(levels)


Key: 0Rep
Num Packets: 5
[  1.  15.   0.   0.   0.   0.   0.   0.]
('0.0', False)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
[ 1.  7.  8.  0.  0.  0.  0.  0.]
('0.0', True)
Key: 1Rep
Num Packets: 5
[  1.  14.   0.   0.   0.   1.   0.   0.]
('1.0', False)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
[ 1.  6.  8.  0.  0.  1.  0.  0.]
('1.0', True)
Key: 2Rep
Num Packets: 5
[  1.  13.   0.   0.   0.   2.   0.   0.]
('2.0', False)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
[ 1.  5.  8.  0.  0.  2.  0.  0.]
('2.0', True)
Key: 3Rep
Num Packets: 5
[  1.  12.   0.   0.   0.   3.   0.   0.]
('3.0', False)
[ 1.  4.  8.  0.  0.  3.  0.  0.]
('3.0', True)
[ 1.  4.  8.  0.  0.  3.  0.  0.]
('3.0', True)
